# Investment Research with AgentCore Gateway - Version 2
## Complete Multi-Tool Financial Analysis using AgentCore Gateway Architecture

## Overview
This notebook demonstrates the complete AgentCore Gateway architecture for investment research, where:
- **Supervisor Agent** runs in AgentCore Runtime
- **AgentCore Gateway** exposes 3 specialized tools:
  1. **Alpha Vantage API** (3rd party marketplace tool)
  2. **yfinance Lambda** (stock data + financial news search)
  3. **Bedrock Knowledge Base** (Amazon 10-K reports)

### Architecture
```
AgentCore Runtime (Supervisor Agent)
           ↓
    AgentCore Gateway
           ↓
┌─────────────────────────────────────────┐
│ Tool 1: Alpha Vantage (Marketplace)     │
│ Tool 2: yfinance Lambda (Stock + News)  │
│ Tool 3: Bedrock KB (Amazon 10-K)        │
└─────────────────────────────────────────┘
```

## Step 1: Install Dependencies and Setup

In [ ]:
# Install required packages
!pip install --force-reinstall -U boto3 requests beautifulsoup4 yfinance pandas numpy --quiet

# Import required libraries
import os
import boto3
import json
import time
import zipfile
import subprocess
import shutil
import requests
import uuid
from datetime import datetime
from botocore.exceptions import ClientError

# Configure AWS region
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')
REGION = os.environ['AWS_DEFAULT_REGION']

print(f"✅ AWS Region configured: {REGION}")

# Verify AWS access
try:
    sts = boto3.client('sts')
    identity = sts.get_caller_identity()
    print(f"✅ AWS Identity: {identity['Arn']}")
except Exception as e:
    print(f"❌ AWS credentials not configured: {e}")

## Step 2: Create Financial Tools Lambda Function

This Lambda function contains:
1. **Stock Data Lookup** (yfinance) - from original notebook
2. **Financial News Search** (web scraping) - from original notebook

In [ ]:
# Create the Lambda function code with financial tools
lambda_code = '''
import json
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import urllib.parse
from datetime import datetime
from typing import Dict, List, Union
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

def stock_data_lookup(ticker: str, period: str = "1mo") -> Dict:
    """Gets historical stock price data for a given stock ticker."""
    try:
        logger.info(f"Fetching stock data for {ticker} with period {period}")
        
        if not ticker.strip():
            return {"status": "error", "message": "Ticker symbol is required"}

        stock = yf.Ticker(ticker)
        data = stock.history(period=period)
        
        if data.empty:
            return {"status": "error", "message": f"No data found for ticker {ticker}"}

        # Calculate key metrics
        current_price = float(data["Close"].iloc[-1])
        previous_close = float(data["Close"].iloc[-2]) if len(data) > 1 else current_price
        price_change = current_price - previous_close
        price_change_percent = (price_change / previous_close) * 100 if previous_close != 0 else 0
        
        # Calculate volatility
        returns = data["Close"].pct_change().dropna()
        volatility = returns.std() * np.sqrt(252) * 100  # Annualized volatility
        
        return {
            "status": "success",
            "data": {
                "symbol": ticker.upper(),
                "current_price": round(current_price, 2),
                "previous_close": round(previous_close, 2),
                "price_change": round(price_change, 2),
                "price_change_percent": round(price_change_percent, 2),
                "volume": int(data["Volume"].iloc[-1]),
                "high": round(float(data["High"].max()), 2),
                "low": round(float(data["Low"].min()), 2),
                "volatility": round(volatility, 2),
                "data_points": len(data),
                "period": period,
                "date": datetime.now().strftime("%Y-%m-%d")
            }
        }
    except Exception as e:
        logger.error(f"Error fetching stock data for {ticker}: {str(e)}")
        return {"status": "error", "message": f"Error fetching price data: {str(e)}"}

def financial_news_search(query: str, max_results: int = 5) -> Dict:
    """Performs web search for financial news and information."""
    try:
        logger.info(f"Performing financial news search for: {query}")
        
        # Use DuckDuckGo for web search (no API key required)
        search_url = "https://duckduckgo.com/html/"
        params = {
            'q': f"{query} finance news",
            't': 'h_',
            'ia': 'web'
        }
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        response = requests.get(search_url, params=params, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        results = []
        
        # Extract search results
        for result in soup.find_all('div', class_='result')[:max_results]:
            title_elem = result.find('a', class_='result__a')
            snippet_elem = result.find('a', class_='result__snippet')
            
            if title_elem:
                title = title_elem.get_text(strip=True)
                snippet = snippet_elem.get_text(strip=True) if snippet_elem else ''
                url = title_elem.get('href', '')
                
                # Clean URL
                if url.startswith('/l/?uddg='):
                    url = url.replace('/l/?uddg=', '')
                    url = urllib.parse.unquote(url)
                
                results.append({
                    "title": title,
                    "summary": snippet,
                    "url": url,
                    "source": "DuckDuckGo"
                })
        
        return {
            "status": "success",
            "data": {
                "query": query,
                "results": results,
                "count": len(results),
                "date": datetime.now().strftime("%Y-%m-%d")
            }
        }
        
    except Exception as e:
        logger.error(f"Error in financial news search: {str(e)}")
        return {"status": "error", "message": f"Error in web search: {str(e)}"}

def amazon_10k_knowledge_base_tool(query: str, max_results: int = 5):
    """Real Bedrock Knowledge Base integration."""
    try:
        import boto3
        
        # Initialize Bedrock client
        bedrock_client = boto3.client('bedrock-agent-runtime', region_name='us-east-1')
        
        # Knowledge Base ID
        kb_id = "0VKIAF6ZOO"
        
        # Query the knowledge base
        response = bedrock_client.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={
                'text': query
            },
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results
                }
            }
        )
        
        # Process results
        results = []
        for result in response.get('retrievalResults', []):
            results.append({
                "title": f"Amazon 10-K Document",
                "content": result.get('content', {}).get('text', ''),
                "score": result.get('score', 0),
                "source": result.get('location', {}).get('s3Location', {}).get('uri', 'Knowledge Base'),
                "metadata": result.get('metadata', {})
            })
        
        return {
            "status": "success",
            "data": {
                "query": query,
                "results": results,
                "count": len(results),
                "knowledge_base_id": kb_id,
                "source": "Bedrock Knowledge Base"
            }
        }
        
    except Exception as e:
        return {"status": "error", "message": f"KB Error: {str(e)}"}

def lambda_handler(event, context):
    """Main Lambda handler that routes requests to appropriate tools."""
    try:
        logger.info(f"Lambda invoked with event: {json.dumps(event)}")
        
        # Extract tool name and arguments from the event
        tool_name = event.get('tool_name')
        arguments = event.get('arguments', {})
        
        if not tool_name:
            return {
                "statusCode": 400,
                "body": json.dumps({"status": "error", "message": "tool_name is required"})
            }
        
        # Route to appropriate tool function
        if tool_name == 'stock_data_lookup_tool':
            result = stock_data_lookup(**arguments)
        elif tool_name == 'financial_news_search_tool':
            result = financial_news_search(**arguments)
        elif tool_name == 'amazon_10k_knowledge_base_tool':
            result = amazon_10k_knowledge_base_tool(**arguments)
        else:
            result = {"status": "error", "message": f"Unknown tool: {tool_name}"}
        
        return {
            "statusCode": 200,
            "body": json.dumps(result)
        }
        
    except Exception as e:
        logger.error(f"Lambda handler error: {str(e)}")
        return {
            "statusCode": 500,
            "body": json.dumps({"status": "error", "message": f"Lambda execution error: {str(e)}"})
        }
'''

# Write the Lambda code to a file
with open('ver2_financial_tools_lambda.py', 'w') as f:
    f.write(lambda_code)

print("✅ Financial Tools Lambda function code created")
print("📁 File: ver2_financial_tools_lambda.py")
print("🔧 Tools included:")
print("   - stock_data_lookup_tool (yfinance)")
print("   - financial_news_search_tool (web scraping)")
print("   - amazon_10k_knowledge_base_tool (real Bedrock KB integration)")

## Step 3: Create Lambda Deployment Package with Dependencies

In [ ]:
def create_lambda_package_with_dependencies():
    """Create a deployment package for the Lambda function with all dependencies."""
    print("📦 Creating Lambda deployment package with dependencies...")
    
    # Create a temporary directory for the package
    package_dir = 'ver2_lambda_package'
    if os.path.exists(package_dir):
        shutil.rmtree(package_dir)
    os.makedirs(package_dir)
    
    # Copy the Lambda function code
    shutil.copy('ver2_financial_tools_lambda.py', os.path.join(package_dir, 'lambda_function.py'))
    
    # Create comprehensive requirements.txt for financial tools
    requirements = '''
yfinance==0.2.65
pandas==2.2.3
numpy==1.26.4
requests==2.32.4
beautifulsoup4==4.13.4
lxml==5.4.0
multitasking==0.0.12
'''
    
    with open('ver2_requirements.txt', 'w') as f:
        f.write(requirements.strip())
    
    print("📥 Installing dependencies...")
    
    # Create the ZIP file
    zip_filename = 'ver2_financial_tools_lambda.zip'
    
    print("📦 Creating ZIP package...")
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(package_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, package_dir)
                zipf.write(file_path, arcname)
    
    # Get file size
    file_size = os.path.getsize(zip_filename) / (1024 * 1024)  # MB
    
    print(f"✅ Lambda package created: {zip_filename}")
    print(f"📊 Package size: {file_size:.2f} MB")
    
    # Clean up
    shutil.rmtree(package_dir)
    
    return zip_filename

def create_lambda_execution_role():
    """Create IAM role for Lambda function execution."""
    iam = boto3.client('iam')
    role_name = 'ver2-financial-tools-lambda-role'
    
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {"Service": "lambda.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }]
    }
    
    try:
        response = iam.get_role(RoleName=role_name)
        print(f"✅ IAM role already exists: {role_name}")
        role_arn = response['Role']['Arn']
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print(f"🔧 Creating IAM role: {role_name}")
            response = iam.create_role(
                RoleName=role_name,
                AssumeRolePolicyDocument=json.dumps(trust_policy),
                Description='IAM role for Financial Tools Lambda function V2'
            )
            role_arn = response['Role']['Arn']
            
            # Attach basic execution policy
            iam.attach_role_policy(
                RoleName=role_name,
                PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
            )
            print(f"✅ IAM role created: {role_arn}")
        else:
            raise e
    
    return role_arn

# Create deployment package and roles
lambda_zip_file = create_lambda_package_with_dependencies()
lambda_role_arn = create_lambda_execution_role()

print(f"\n🎯 Lambda Role ARN: {lambda_role_arn}")

# Wait for IAM propagation
print("⏳ Waiting for IAM role propagation...")
time.sleep(10)
print("✅ IAM roles ready")

## Step 4: Deploy Lambda Function

In [ ]:
def deploy_lambda_function(zip_file, role_arn):
    """Deploy the Lambda function to AWS."""
    lambda_client = boto3.client('lambda')
    function_name = 'ver2-financial-tools-function'
    
    with open(zip_file, 'rb') as f:
        zip_content = f.read()
    
    try:
        # Try to update existing function
        response = lambda_client.get_function(FunctionName=function_name)
        print(f"🔄 Updating existing Lambda function: {function_name}")
        
        response = lambda_client.update_function_code(
            FunctionName=function_name,
            ZipFile=zip_content
        )
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print(f"🔧 Creating new Lambda function: {function_name}")
            
            response = lambda_client.create_function(
                FunctionName=function_name,
                Runtime='python3.12',
                Role=role_arn,
                Handler='lambda_function.lambda_handler',
                Code={'ZipFile': zip_content},
                Description='Financial Tools V2: Stock data lookup and news search',
                Timeout=300,
                MemorySize=1024
            )
        else:
            raise e
    
    function_arn = response['FunctionArn']
    print(f"✅ Lambda function deployed: {function_arn}")
    return function_arn

# Deploy the Lambda function
lambda_function_arn = deploy_lambda_function(lambda_zip_file, lambda_role_arn)
print(f"\n🎯 Financial Tools Lambda Function ARN: {lambda_function_arn}")

## Step 4b: Add Bedrock Knowledge Base Permissions to Lambda

Add the necessary permissions for Lambda to access Bedrock Knowledge Base.

In [ ]:
def add_bedrock_kb_permissions():
    """Add Bedrock KB permissions to Lambda role."""
    
    print("🔐 Adding Bedrock Knowledge Base permissions to Lambda role...")
    
    iam = boto3.client('iam')
    role_name = 'ver2-financial-tools-lambda-role'
    
    # Bedrock policy for KB access
    bedrock_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock:Retrieve",
                    "bedrock:RetrieveAndGenerate"
                ],
                "Resource": kb_config['knowledge_base_arn']
            }
        ]
    }
    
    policy_name = 'ver2-lambda-bedrock-kb-policy'
    
    try:
        # Create policy
        iam.create_policy(
            PolicyName=policy_name,
            PolicyDocument=json.dumps(bedrock_policy),
            Description='Bedrock KB access for Lambda'
        )
        policy_arn = f"arn:aws:iam::{identity['Account']}:policy/{policy_name}"
        print(f"✅ Policy created: {policy_name}")
    except ClientError as e:
        if 'EntityAlreadyExists' in str(e):
            policy_arn = f"arn:aws:iam::{identity['Account']}:policy/{policy_name}"
            print(f"✅ Policy exists: {policy_name}")
        else:
            print(f"❌ Policy error: {e}")
            return
    
    try:
        # Attach policy to role
        iam.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )
        print(f"✅ Policy attached to role: {role_name}")
    except Exception as e:
        print(f"⚠️  Policy attachment: {e}")

# Add Bedrock permissions
add_bedrock_kb_permissions()
print("✅ Lambda function now has access to Bedrock Knowledge Base!")


## Step 5: Set Up Alpha Vantage API (Marketplace Tool)

Configure Alpha Vantage as our 3rd party marketplace tool for professional financial data.

In [ ]:
# Alpha Vantage API setup
print("📈 Setting up Alpha Vantage API (Marketplace Tool)...")

# Note: You'll need to get a free API key from https://www.alphavantage.co/support/#api-key
ALPHA_VANTAGE_API_KEY = "O536HTK78MB2XJI9"  # Replace with your actual API key

def test_alpha_vantage_api(api_key="O536HTK78MB2XJI9"):
    """Test Alpha Vantage API connection."""
    try:
        # Test with demo key (limited functionality)
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey={api_key}"
        
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        
        if "Error Message" in data:
            return {"status": "error", "message": data["Error Message"]}
        elif "Note" in data:
            return {"status": "warning", "message": "API call frequency limit reached"}
        elif "Time Series (5min)" in data:
            return {"status": "success", "message": "Alpha Vantage API working", "data_points": len(data["Time Series (5min)"])}
        else:
            return {"status": "success", "message": "Alpha Vantage API accessible", "response_keys": list(data.keys())}
            
    except Exception as e:
        return {"status": "error", "message": f"Alpha Vantage API error: {str(e)}"}

# Test Alpha Vantage connection
alpha_vantage_test = test_alpha_vantage_api(ALPHA_VANTAGE_API_KEY)

if alpha_vantage_test["status"] == "success":
    print(f"✅ Alpha Vantage API: {alpha_vantage_test['message']}")
elif alpha_vantage_test["status"] == "warning":
    print(f"⚠️  Alpha Vantage API: {alpha_vantage_test['message']}")
else:
    print(f"❌ Alpha Vantage API: {alpha_vantage_test['message']}")

print("\n💡 To get full Alpha Vantage functionality:")
print("   1. Visit: https://www.alphavantage.co/support/#api-key")
print("   2. Get your free API key")
print("   3. Replace 'demo' with your actual API key")
print("\n📊 Alpha Vantage provides:")
print("   - Real-time and historical stock data")
print("   - Technical indicators")
print("   - Fundamental data")
print("   - Economic indicators")
print("   - Cryptocurrency data")

## Step 6: Set Up Bedrock Knowledge Base for Amazon 10-K Reports

Create a Bedrock Knowledge Base using the Amazon 10-K reports you uploaded to S3.

In [ ]:
def setup_bedrock_knowledge_base():
    """Set up Bedrock Knowledge Base for Amazon 10-K reports."""
    
    print("📚 Setting up Bedrock Knowledge Base for Amazon 10-K reports...")
    
    # Knowledge Base configuration
    kb_name = "ver2-amazon-10k-knowledge-base"
    s3_bucket = "ver2-amazon-10k-kb-bucket"  # Your bucket with 10-K reports
    REGION = "us-east-1"
    
    try:
        bedrock_agent = boto3.client('bedrock-agent', region_name=REGION)
        
        # Check if knowledge base already exists
        response = bedrock_agent.list_knowledge_bases()
        existing_kb = None
        
        for kb in response.get('knowledgeBaseSummaries', []):
            if kb['name'] == kb_name:
                existing_kb = kb
                break
        
        if existing_kb:
            print(f"✅ Knowledge Base already exists: {kb_name}")
            kb_id = existing_kb['knowledgeBaseId']
            # Get full KB details to ensure we have the ARN
            try:
                kb_details = bedrock_agent.get_knowledge_base(knowledgeBaseId=kb_id)
                kb_arn = kb_details['knowledgeBase']['knowledgeBaseArn']
                print(f"✅ Knowledge Base ARN retrieved: {kb_arn}")
            except Exception as arn_error:
                print(f"⚠️  Could not get KB ARN, using constructed ARN: {arn_error}")
                # Construct ARN if we can't get it directly
                try:
                    sts_client = boto3.client('sts')
                    account_id = sts_client.get_caller_identity().get('Account', '123456789012')
                except:
                    account_id = '123456789012'
                kb_arn = f'arn:aws:bedrock:{REGION}:{account_id}:knowledge-base/{kb_id}'

        else:
            print(f"💡 Knowledge Base setup requires additional configuration")
            print(f"   - OpenSearch Serverless collection")
            print(f"   - Proper IAM permissions")
            print(f"   - Vector embeddings setup")
            
            # For demo purposes, return placeholder values
            kb_id = 'ver2-demo-kb-id'
            kb_arn = f'arn:aws:bedrock:{REGION}:123456789012:knowledge-base/ver2-demo-kb-id'
        
        return {
            'knowledge_base_id': kb_id,
            'knowledge_base_arn': kb_arn,
            'status': 'ready' if existing_kb else 'demo',
            's3_bucket': s3_bucket
        }
        
    except Exception as e:
        print(f"❌ Error setting up Knowledge Base: {e}")
        print("💡 Note: Knowledge Base setup requires additional permissions")
        return {
            'knowledge_base_id': 'ver2-demo-kb-id',
            'knowledge_base_arn': f'arn:aws:bedrock:{REGION}:123456789012:knowledge-base/ver2-demo-kb-id',
            'status': 'demo',
            's3_bucket': s3_bucket
        }

# Set up Knowledge Base
kb_config = setup_bedrock_knowledge_base()

print(f"\n📚 Knowledge Base Configuration:")
print(f"   ID: {kb_config['knowledge_base_id']}")
print(f"   Status: {kb_config['status']}")
print(f"   S3 Bucket: {kb_config['s3_bucket']}")

## Step 7: Create Amazon Cognito Pool for Authentication

Set up Amazon Cognito User Pool for secure authentication to the AgentCore Gateway.

In [ ]:
def setup_cognito_authentication():
    """Set up Cognito user pool, resource server, and client for authentication."""
    cognito = boto3.client("cognito-idp", region_name=REGION)
    
    USER_POOL_NAME = "ver2-financial-research-gateway-pool"
    RESOURCE_SERVER_ID = "ver2-financial-research-gateway-id"
    CLIENT_NAME = "ver2-financial-research-gateway-client"
    
    # Create or get user pool
    try:
        pools = cognito.list_user_pools(MaxResults=50)['UserPools']
        user_pool = next((p for p in pools if p['Name'] == USER_POOL_NAME), None)
        
        if user_pool:
            user_pool_id = user_pool['Id']
            print(f"✅ Found existing user pool: {USER_POOL_NAME}")
        else:
            print(f"🔧 Creating user pool: {USER_POOL_NAME}")
            response = cognito.create_user_pool(
                PoolName=USER_POOL_NAME,
                Policies={'PasswordPolicy': {'MinimumLength': 8}}
            )
            user_pool_id = response['UserPool']['Id']
            print(f"✅ User pool created: {user_pool_id}")
    except Exception as e:
        print(f"❌ Error with user pool: {e}")
        raise
    
    # Create or get resource server
    try:
        try:
            cognito.describe_resource_server(
                UserPoolId=user_pool_id,
                Identifier=RESOURCE_SERVER_ID
            )
            print(f"✅ Resource server exists: {RESOURCE_SERVER_ID}")
        except ClientError as e:
            if e.response['Error']['Code'] == 'ResourceNotFoundException':
                print(f"🔧 Creating resource server: {RESOURCE_SERVER_ID}")
                cognito.create_resource_server(
                    UserPoolId=user_pool_id,
                    Identifier=RESOURCE_SERVER_ID,
                    Name="Ver2 Financial Research Gateway",
                    Scopes=[
                        {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
                        {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
                    ]
                )
                print(f"✅ Resource server created")
    except Exception as e:
        print(f"❌ Error with resource server: {e}")
        raise
    
    # Create or get M2M client
    try:
        clients = cognito.list_user_pool_clients(UserPoolId=user_pool_id)['UserPoolClients']
        client = next((c for c in clients if c['ClientName'] == CLIENT_NAME), None)
        
        if client:
            client_id = client['ClientId']
            client_details = cognito.describe_user_pool_client(
                UserPoolId=user_pool_id,
                ClientId=client_id
            )
            client_secret = client_details['UserPoolClient'].get('ClientSecret')
            print(f"✅ Found existing client: {CLIENT_NAME}")
        else:
            print(f"🔧 Creating M2M client: {CLIENT_NAME}")
            response = cognito.create_user_pool_client(
                UserPoolId=user_pool_id,
                ClientName=CLIENT_NAME,
                GenerateSecret=True,
                AllowedOAuthFlows=['client_credentials'],
                AllowedOAuthScopes=[
                    f"{RESOURCE_SERVER_ID}/gateway:read",
                    f"{RESOURCE_SERVER_ID}/gateway:write"
                ],
                AllowedOAuthFlowsUserPoolClient=True
            )
            client_id = response['UserPoolClient']['ClientId']
            client_secret = response['UserPoolClient']['ClientSecret']
            print(f"✅ M2M client created: {client_id}")
    except Exception as e:
        print(f"❌ Error with M2M client: {e}")
        raise
    
    # Create OAuth domain
    domain_name = f"ver2-financial-research-{str(uuid.uuid4())[:8]}"
    try:
        cognito.create_user_pool_domain(
            Domain=domain_name,
            UserPoolId=user_pool_id
        )
        print(f"✅ OAuth domain created: {domain_name}")
    except ClientError as e:
        if e.response['Error']['Code'] == 'InvalidParameterException':
            print(f"⚠️  Domain might already exist or invalid: {domain_name}")
        else:
            print(f"❌ Error creating domain: {e}")
    
    # Discovery URL and token endpoint
    discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
    token_endpoint = f'https://{domain_name}.auth.{REGION}.amazoncognito.com/oauth2/token'
    scope_string = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"
    
    return {
        'user_pool_id': user_pool_id,
        'client_id': client_id,
        'client_secret': client_secret,
        'discovery_url': discovery_url,
        'token_endpoint': token_endpoint,
        'scope_string': scope_string,
        'resource_server_id': RESOURCE_SERVER_ID,
        'domain_name': domain_name
    }

# Set up Cognito authentication
print("🔐 Setting up Amazon Cognito authentication...")
cognito_config = setup_cognito_authentication()

print(f"\n📋 Cognito Configuration:")
print(f"   User Pool ID: {cognito_config['user_pool_id']}")
print(f"   Client ID: {cognito_config['client_id']}")
print(f"   Domain: {cognito_config['domain_name']}")
print(f"   Token Endpoint: {cognito_config['token_endpoint']}")
print("✅ Amazon Cognito setup completed!")

## Step 8: Create IAM Role for AgentCore Gateway

In [ ]:
def create_agentcore_gateway_role():
    """Create IAM role for AgentCore Gateway with necessary permissions."""
    iam = boto3.client('iam')
    role_name = 'ver2-AgentCore-Gateway-ExecutionRole'
    
    # Trust policy for AgentCore Gateway
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    # Policy for Lambda invocation and Bedrock access
    gateway_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "lambda:InvokeFunction"
                ],
                "Resource": lambda_function_arn
            },
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock:Retrieve",
                    "bedrock:RetrieveAndGenerate"
                ],
                "Resource": kb_config['knowledge_base_arn']
            },
            {
                "Effect": "Allow",
                "Action": [
                    "logs:CreateLogGroup",
                    "logs:CreateLogStream",
                    "logs:PutLogEvents"
                ],
                "Resource": "arn:aws:logs:*:*:*"
            }
        ]
    }
    
    try:
        # Check if role exists
        response = iam.get_role(RoleName=role_name)
        print(f"✅ Gateway IAM role already exists: {role_name}")
        role_arn = response['Role']['Arn']
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print(f"🔧 Creating Gateway IAM role: {role_name}")
            
            # Create role
            response = iam.create_role(
                RoleName=role_name,
                AssumeRolePolicyDocument=json.dumps(trust_policy),
                Description='IAM role for AgentCore Gateway V2'
            )
            role_arn = response['Role']['Arn']
            
            # Create and attach policy
            policy_name = 'ver2-AgentCore-Gateway-Policy'
            try:
                iam.create_policy(
                    PolicyName=policy_name,
                    PolicyDocument=json.dumps(gateway_policy),
                    Description='Policy for AgentCore Gateway V2'
                )
                policy_arn = f"arn:aws:iam::{identity['Account']}:policy/{policy_name}"
            except ClientError as policy_error:
                if policy_error.response['Error']['Code'] == 'EntityAlreadyExists':
                    policy_arn = f"arn:aws:iam::{identity['Account']}:policy/{policy_name}"
                else:
                    raise policy_error
            
            # Attach policy to role
            iam.attach_role_policy(
                RoleName=role_name,
                PolicyArn=policy_arn
            )
            
            print(f"✅ Gateway IAM role created: {role_arn}")
        else:
            raise e
    
    return role_arn

# Create Gateway IAM role
gateway_role_arn = create_agentcore_gateway_role()
print(f"\n🎯 Gateway Role ARN: {gateway_role_arn}")

# Wait for IAM propagation
print("⏳ Waiting for IAM role propagation...")
time.sleep(15)
print("✅ Gateway IAM role ready")

## Step 9: Create AgentCore Gateway with MCP Tools

Create the AgentCore Gateway that exposes our 3 tools via MCP protocol.

In [ ]:
def create_agentcore_gateway():
    """Create AgentCore Gateway with MCP-compliant tool definitions."""
    
    gateway_name = "ver2-financial-research-gateway"
    
    try:
        # Use the correct client from the working example
        gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
        print("🔧 Using bedrock-agentcore-control client")
        
        # Auth configuration using Cognito (from working example)
        auth_config = {
            "customJWTAuthorizer": { 
                "allowedClients": [cognito_config['client_id']],
                "discoveryUrl": cognito_config['discovery_url']
            }
        }
        
        # Create gateway using the working method signature
        create_response = gateway_client.create_gateway(
            name=gateway_name,
            roleArn=gateway_role_arn,
            protocolType='MCP',
            authorizerType='CUSTOM_JWT',
            authorizerConfiguration=auth_config,
            description='Financial Research Gateway V2 with Alpha Vantage, yfinance Lambda, and Bedrock Knowledge Base'
        )
        
        print(f"✅ AgentCore Gateway created successfully!")
        gateway_id = create_response["gatewayId"]
        gateway_url = create_response["gatewayUrl"]
        
        print(f"   Gateway ID: {gateway_id}")
        print(f"   Gateway URL: {gateway_url}")
        
        return {
            'gateway_id': gateway_id,
            'gateway_arn': f'arn:aws:bedrock:{REGION}:123456789012:gateway/{gateway_id}',
            'gateway_name': gateway_name,
            'gateway_url': gateway_url,
            'tools_count': 3,
            'status': 'active'
        }
        
    except Exception as e:
        print(f"❌ Error creating AgentCore Gateway: {e}")
        print("💡 Falling back to demo mode")
        
        return {
            'gateway_id': 'ver2-demo-gateway-id',
            'gateway_arn': f'arn:aws:bedrock:{REGION}:123456789012:agent-gateway/ver2-demo-gateway-id',
            'gateway_name': gateway_name,
            'tools_count': 3,
            'status': 'demo'
        }

# Create AgentCore Gateway
print("🚪 Creating AgentCore Gateway...")
gateway_config = create_agentcore_gateway()

print(f"\n🎯 AgentCore Gateway Configuration:")
print(f"   Gateway ID: {gateway_config['gateway_id']}")
print(f"   Gateway Name: {gateway_config['gateway_name']}")
print(f"   Tools Count: {gateway_config['tools_count']}")
print(f"   Status: {gateway_config.get('status', 'active')}")
if 'gateway_url' in gateway_config:
    print(f"   Gateway URL: {gateway_config['gateway_url']}")
print("✅ AgentCore Gateway setup completed!")


## Step 9b: Gateway Target Creation

In [ ]:
def create_gateway_target():
    """Create Gateway Target with Lambda and tool specifications."""
    
    if gateway_config['status'] == 'demo':
        print("⚠️  Gateway is in demo mode, skipping target creation")
        return {'status': 'demo'}
    
    try:
        gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
        
        # Lambda target configuration with tool schema (from working example)
        lambda_target_config = {
            "mcp": {
                "lambda": {
                    "lambdaArn": lambda_function_arn,
                    "toolSchema": {
                        "inlinePayload": [
                            {
                                "name": "stock_data_lookup_tool",
                                "description": "Gets historical stock price data, current price, volume, and volatility metrics for a given stock ticker symbol using yfinance.",
                                "inputSchema": {
                                    "type": "object",
                                    "properties": {
                                        "ticker": {
                                            "type": "string",
                                            "description": "Stock ticker symbol (e.g., AAPL, GOOGL, TSLA)"
                                        },
                                        "period": {
                                            "type": "string",
                                            "description": "Time period for historical data (1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max)"
                                        }
                                    },
                                    "required": ["ticker"]
                                }
                            },
                            {
                                "name": "financial_news_search_tool",
                                "description": "Performs web search for financial news and information related to stocks, companies, or market topics.",
                                "inputSchema": {
                                    "type": "object",
                                    "properties": {
                                        "query": {
                                            "type": "string",
                                            "description": "Search query for financial news (e.g., 'Apple earnings', 'Tesla stock news', 'market volatility')"
                                        },
                                        "max_results": {
                                            "type": "integer",
                                            "description": "Maximum number of search results to return (1-10)"
                                        }
                                    },
                                    "required": ["query"]
                                }
                            },
                            {
                                "name": "amazon_10k_knowledge_base_tool",
                                "description": "Searches Amazon 10-K reports and financial documents using Bedrock Knowledge Base for detailed company financial information.",
                                "inputSchema": {
                                    "type": "object",
                                    "properties": {
                                        "query": {
                                            "type": "string",
                                            "description": "Search query for Amazon financial documents (e.g., 'revenue growth', 'operating expenses', 'risk factors')"
                                        },
                                        "max_results": {
                                            "type": "integer",
                                            "description": "Maximum number of knowledge base results to return (1-10)"
                                        }
                                    },
                                    "required": ["query"]
                                }
                            }
                        ]
                    }
                }
            }
        }

        # Credential configuration (from working example)
        credential_config = [
            {
                "credentialProviderType": "GATEWAY_IAM_ROLE"
            }
        ]
        
        target_name = 'FinancialToolsTarget'
        
        # Create gateway target
        response = gateway_client.create_gateway_target(
            gatewayIdentifier=gateway_config['gateway_id'],
            name=target_name,
            description='Financial Tools Lambda Target with Stock Data, News Search, and Knowledge Base',
            targetConfiguration=lambda_target_config,
            credentialProviderConfigurations=credential_config
        )
        
        print(f"✅ Gateway Target created successfully!")
        print(f"   Target Name: {target_name}")
        print(f"   Tools Available:")
        print(f"     - stock_data_lookup_tool")
        print(f"     - financial_news_search_tool") 
        print(f"     - amazon_10k_knowledge_base_tool")
        
        return {
            'target_name': target_name,
            'status': 'active',
            'tools_count': 3
        }
        
    except Exception as e:
        print(f"❌ Error creating Gateway Target: {e}")
        return {'status': 'error', 'message': str(e)}

# Create Gateway Target with tool specifications
print("\n🔧 Creating Gateway Target with tool specifications...")
target_config = create_gateway_target()

if target_config['status'] == 'active':
    print("✅ Gateway Target setup completed!")
    print(f"🎯 Your gateway now has {target_config['tools_count']} MCP tools ready to use!")
else:
    print("⚠️  Gateway Target creation skipped or failed")


## Step 9b-prereq: Verify Workshop Prerequisites

Verify all required AWS resources exist before proceeding with Alpha Vantage integration.

In [ ]:
def verify_workshop_prerequisites():
    """Verify all required resources exist for the workshop."""
    
    print("🔍 Verifying Workshop Prerequisites...")
    print("=" * 50)
    
    # Check Gateway
    try:
        gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
        gateways = gateway_client.list_gateways()['items']
        gateway = next((g for g in gateways if 'ver2-financial-research-gateway' in g['name']), None)
        
        if gateway and gateway['status'] == 'READY':
            print(f"✅ Gateway: {gateway['name']} (READY)")
            gateway_config = {
                'gateway_id': gateway['gatewayId'],
                'gateway_name': gateway['name'],
                'status': gateway['status']
            }
            globals()['gateway_config'] = gateway_config
        else:
            print("❌ Gateway: Not found or not ready")
            return False
    except Exception as e:
        print(f"❌ Gateway check failed: {e}")
        return False
    
    print("\n✅ All prerequisites verified! Ready for Alpha Vantage integration.")
    return True

# Verify prerequisites
if verify_workshop_prerequisites():
    print("\n🎉 Workshop environment is ready!")
else:
    print("\n⚠️  Please complete previous steps before proceeding to Alpha Vantage integration.")


## Step 9c: Create Alpha Vantage OpenAPI Target (Working Implementation)

Create Alpha Vantage as a marketplace tool using OpenAPI specification with proper credential provider.

In [ ]:
def create_alpha_vantage_complete_setup():
    """Create Alpha Vantage OpenAPI target with credential provider."""
    
    print("📈 Creating Alpha Vantage OpenAPI target with credential provider...")
    
    client = boto3.client('bedrock-agentcore-control', region_name=REGION)
    
    # Get gateway
    gateways = client.list_gateways()['items']
    gateway = next((g for g in gateways if 'ver2-financial-research-gateway' in g['name']), None)
    
    if not gateway:
        print("❌ Gateway not found")
        return None
    
    gateway_id = gateway['gatewayId']
    
    # Step 1: Create API key credential provider
    print("🔐 Creating API key credential provider...")
    try:
        cred_response = client.create_api_key_credential_provider(
            name='alpha-vantage-api-key',
            apiKey='O536HTK78MB2XJI9'
        )
        provider_arn = cred_response['credentialProviderArn']
        print(f"✅ Credential provider created: {provider_arn}")
    except Exception as e:
        if 'already exists' in str(e).lower():
            # Get existing provider
            providers = client.list_api_key_credential_providers()
            provider = next((p for p in providers['credentialProviders'] if p['name'] == 'alpha-vantage-api-key'), None)
            if provider:
                provider_arn = provider['credentialProviderArn']
                print(f"✅ Using existing credential provider: {provider_arn}")
            else:
                print(f"❌ Credential provider error: {e}")
                return None
        else:
            print(f"❌ Credential provider error: {e}")
            return None
    
    # Step 2: Create OpenAPI spec
    openapi_spec = {
        "openapi": "3.0.0",
        "info": {
            "title": "Alpha Vantage Stock API",
            "version": "1.0.0"
        },
        "servers": [{"url": "https://www.alphavantage.co"}],
        "paths": {
            "/query": {
                "get": {
                    "operationId": "getStockQuote",
                    "summary": "Get stock quote from Alpha Vantage",
                    "parameters": [
                        {
                            "name": "function",
                            "in": "query",
                            "required": True,
                            "schema": {"type": "string", "enum": ["GLOBAL_QUOTE"]},
                            "description": "API function"
                        },
                        {
                            "name": "symbol",
                            "in": "query", 
                            "required": True,
                            "schema": {"type": "string"},
                            "description": "Stock symbol (e.g., AAPL)"
                        }
                    ],
                    "responses": {
                        "200": {
                            "description": "Stock quote data",
                            "content": {
                                "application/json": {
                                    "schema": {"type": "object"}
                                }
                            }
                        }
                    }
                }
            }
        },
        "components": {
            "securitySchemes": {
                "ApiKeyAuth": {
                    "type": "apiKey",
                    "in": "query",
                    "name": "apikey"
                }
            }
        },
        "security": [{"ApiKeyAuth": []}]
    }
    
    # Step 3: Create gateway target
    print("🎯 Creating gateway target...")
    try:
        response = client.create_gateway_target(
            gatewayIdentifier=gateway_id,
            name='alpha-vantage-openapi-target',
            targetConfiguration={
                "mcp": {
                    "openApiSchema": {
                        "inlinePayload": json.dumps(openapi_spec)
                    }
                }
            },
            credentialProviderConfigurations=[
                {
                    "credentialProviderType": "API_KEY",
                    "credentialProvider": {
                        "apiKeyCredentialProvider": {
                            "providerArn": provider_arn,
                            "credentialLocation": "QUERY_PARAMETER",
                            "credentialParameterName": "apikey"
                        }
                    }
                }
            ]
        )
        
        print("✅ Alpha Vantage OpenAPI target created successfully!")
        print(f"   Target ID: {response['gatewayTargetId']}")
        print(f"   Tool: getStockQuote")
        print(f"   Credential Provider: {provider_arn}")
        
        return response['gatewayTargetId']
        
    except Exception as e:
        print(f"❌ Error creating gateway target: {e}")
        return None

# Create Alpha Vantage OpenAPI target
target_id = create_alpha_vantage_complete_setup()

if target_id:
    print("\n🎉 Step 9c Complete - Alpha Vantage OpenAPI Target Created!")
    print("\n📝 Available Tools:")
    print("   • getStockQuote - Get real-time stock quotes from Alpha Vantage")
    print("\n💡 Test with: symbol='AAPL', function='GLOBAL_QUOTE'")
else:
    print("\n❌ Step 9c Failed - OpenAPI target creation unsuccessful")


## Step 10: Test Gateway Authentication

Test the Cognito authentication flow for the AgentCore Gateway.

In [ ]:
def test_cognito_authentication():
    """Test Cognito M2M authentication flow."""
    print("🔐 Testing Cognito authentication...")
    
    try:
        # Prepare authentication request
        token_url = cognito_config['token_endpoint']
        
        # Basic auth header
        import base64
        credentials = f"{cognito_config['client_id']}:{cognito_config['client_secret']}"
        encoded_credentials = base64.b64encode(credentials.encode()).decode()
        
        headers = {
            'Authorization': f'Basic {encoded_credentials}',
            'Content-Type': 'application/x-www-form-urlencoded'
        }
        
        data = {
            'grant_type': 'client_credentials',
            'scope': cognito_config['scope_string']
        }
        
        # Make token request
        response = requests.post(token_url, headers=headers, data=data, timeout=10)
        
        if response.status_code == 200:
            token_data = response.json()
            access_token = token_data.get('access_token')
            token_type = token_data.get('token_type', 'Bearer')
            expires_in = token_data.get('expires_in')
            
            print(f"✅ Authentication successful!")
            print(f"   Token Type: {token_type}")
            print(f"   Expires In: {expires_in} seconds")
            print(f"   Token Preview: {access_token[:50]}...")
            
            return {
                'status': 'success',
                'access_token': access_token,
                'token_type': token_type,
                'expires_in': expires_in
            }
        else:
            print(f"❌ Authentication failed: {response.status_code}")
            print(f"   Response: {response.text}")
            return {'status': 'error', 'message': response.text}
            
    except Exception as e:
        print(f"❌ Authentication error: {e}")
        return {'status': 'error', 'message': str(e)}

# Test authentication
auth_test = test_cognito_authentication()

if auth_test['status'] == 'success':
    print("\n🎉 Authentication test passed!")
    print("   The gateway is ready to accept authenticated requests")
else:
    print("\n⚠️  Authentication test failed")
    print("   Check Cognito configuration and domain setup")

## Step 11: Set Up Strands Agent for MCP Tool Testing

Create a Strands Agent to test our AgentCore Gateway tools via MCP protocol.

In [ ]:
def create_agent_execution_role():
    """Create IAM role for Strands Agent execution."""
    iam = boto3.client('iam')
    role_name = 'ver2-strands-agent-execution-role'
    
    # Trust policy for Bedrock Agent
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    # Policy for agent operations
    agent_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock:InvokeModel",
                    "bedrock:InvokeModelWithResponseStream"
                ],
                "Resource": "*"
            },
            {
                "Effect": "Allow",
                "Action": [
                    "logs:CreateLogGroup",
                    "logs:CreateLogStream",
                    "logs:PutLogEvents"
                ],
                "Resource": "arn:aws:logs:*:*:*"
            }
        ]
    }
    
    try:
        response = iam.get_role(RoleName=role_name)
        print(f"✅ Agent IAM role already exists: {role_name}")
        role_arn = response['Role']['Arn']
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print(f"🔧 Creating Agent IAM role: {role_name}")
            response = iam.create_role(
                RoleName=role_name,
                AssumeRolePolicyDocument=json.dumps(trust_policy),
                Description='IAM role for Strands Agent V2'
            )
            role_arn = response['Role']['Arn']
            
            # Create and attach policy
            policy_name = 'ver2-strands-agent-policy'
            try:
                iam.create_policy(
                    PolicyName=policy_name,
                    PolicyDocument=json.dumps(agent_policy),
                    Description='Policy for Strands Agent V2'
                )
                policy_arn = f"arn:aws:iam::{identity['Account']}:policy/{policy_name}"
            except ClientError as policy_error:
                if policy_error.response['Error']['Code'] == 'EntityAlreadyExists':
                    policy_arn = f"arn:aws:iam::{identity['Account']}:policy/{policy_name}"
                else:
                    raise policy_error
            
            # Attach policy to role
            iam.attach_role_policy(
                RoleName=role_name,
                PolicyArn=policy_arn
            )
            
            print(f"✅ Agent IAM role created: {role_arn}")
        else:
            raise e
    
    return role_arn

def create_strands_agent_for_testing():
    """Create a Strands Agent to test the AgentCore Gateway tools."""
    
    # First create the IAM role
    agent_role_arn = create_agent_execution_role()
    
    # Wait for IAM propagation
    print("⏳ Waiting for IAM role propagation...")
    time.sleep(10)
    
    bedrock_agent = boto3.client('bedrock-agent', region_name=REGION)
    agent_name = "ver2-financial-research-strands-agent"
    
    # Agent configuration with IAM role
    agent_config = {
        "agentName": agent_name,
        "agentResourceRoleArn": agent_role_arn,  # This was missing!
        "description": "Strands Agent V2 for testing Financial Research Gateway MCP tools",
        "instruction": """You are a financial research assistant with access to three specialized tools:

1. **stock_data_lookup_tool**: Get real-time stock data, prices, and volatility metrics
2. **financial_news_search_tool**: Search for financial news and market information
3. **amazon_10k_knowledge_base_tool**: Query Amazon's 10-K reports and financial documents

Use these tools to provide comprehensive financial analysis and investment insights. Always cite your sources and provide data-driven recommendations.""",
        "foundationModel": "anthropic.claude-3-sonnet-20240229-v1:0",
        "idleSessionTTLInSeconds": 1800
    }
    
    try:
        # Check if agent already exists
        agents = bedrock_agent.list_agents()
        existing_agent = None
        
        for agent in agents.get('agentSummaries', []):
            if agent['agentName'] == agent_name:
                existing_agent = agent
                break
        
        if existing_agent:
            print(f"✅ Strands Agent already exists: {agent_name}")
            agent_id = existing_agent['agentId']
            agent_arn = existing_agent['agentArn']
        else:
            print(f"🔧 Creating Strands Agent: {agent_name}")
            response = bedrock_agent.create_agent(**agent_config)
            
            agent_id = response['agent']['agentId']
            agent_arn = response['agent']['agentArn']
            
            print(f"✅ Strands Agent created: {agent_id}")
        
        return {
            'agent_id': agent_id,
            'agent_arn': agent_arn,
            'agent_name': agent_name,
            'agent_role_arn': agent_role_arn,
            'status': 'ready'
        }
        
    except Exception as e:
        print(f"❌ Error creating Strands Agent: {e}")
        return {
            'agent_id': 'ver2-demo-agent-id',
            'agent_arn': f'arn:aws:bedrock:{REGION}:123456789012:agent/ver2-demo-agent-id',
            'agent_name': agent_name,
            'status': 'demo'
        }

def test_strands_agent_tools():
    """Test the Strands Agent with sample queries."""
    print("🧪 Testing Strands Agent with sample queries...")
    
    test_queries = [
        "Get the current stock price and recent performance for Apple (AAPL)",
        "Search for recent news about Tesla stock performance",
        "What does Amazon's 10-K report say about their revenue growth?"
    ]
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n📝 Test Query {i}: {query}")
        print(f"   Expected Tool: {['stock_data_lookup_tool', 'financial_news_search_tool', 'amazon_10k_knowledge_base_tool'][i-1]}")
        print(f"   Status: Ready for testing")
    
    print("\n💡 To test the agent:")
    print("   1. Use the Bedrock console to invoke the agent")
    print("   2. Try the sample queries above")
    print("   3. Verify each tool responds correctly")

# Create Strands Agent with proper IAM role
print("🤖 Setting up Strands Agent for testing...")
agent_config = create_strands_agent_for_testing()

print(f"\n🎯 Strands Agent Configuration:")
print(f"   Agent ID: {agent_config['agent_id']}")
print(f"   Agent Name: {agent_config['agent_name']}")
print(f"   Status: {agent_config['status']}")
if 'agent_role_arn' in agent_config:
    print(f"   IAM Role: {agent_config['agent_role_arn']}")

# Test agent setup
test_strands_agent_tools()
print("\n✅ Strands Agent setup completed!")


In [ ]:
## Prepare Your Agent
def prepare_strands_agent():
    """Prepare the Strands Agent for use."""
    
    try:
        bedrock_agent = boto3.client('bedrock-agent', region_name=REGION)
        agent_id = agent_config['agent_id']
        
        print(f"🔧 Preparing Strands Agent: {agent_id}")
        
        # Prepare the agent
        response = bedrock_agent.prepare_agent(agentId=agent_id)
        
        print(f"✅ Agent preparation initiated!")
        print(f"   Agent ID: {agent_id}")
        print(f"   Status: {response['agentStatus']}")
        print(f"   Prepared At: {response.get('preparedAt', 'In Progress')}")
        
        # Wait for preparation to complete
        print("⏳ Waiting for agent preparation to complete...")
        
        max_attempts = 30
        attempt = 0
        
        while attempt < max_attempts:
            try:
                agent_details = bedrock_agent.get_agent(agentId=agent_id)
                status = agent_details['agent']['agentStatus']
                
                print(f"   Attempt {attempt + 1}: Status = {status}")
                
                if status == 'PREPARED':
                    print("✅ Agent is now PREPARED and ready to use!")
                    return {
                        'status': 'prepared',
                        'agent_id': agent_id,
                        'agent_status': status
                    }
                elif status == 'FAILED':
                    print("❌ Agent preparation failed!")
                    return {
                        'status': 'failed',
                        'agent_id': agent_id,
                        'agent_status': status
                    }
                
                time.sleep(10)  # Wait 10 seconds before checking again
                attempt += 1
                
            except Exception as e:
                print(f"   Error checking status: {e}")
                time.sleep(5)
                attempt += 1
        
        print("⚠️  Agent preparation is taking longer than expected")
        return {
            'status': 'timeout',
            'agent_id': agent_id,
            'message': 'Preparation timeout - check AWS console'
        }
        
    except Exception as e:
        print(f"❌ Error preparing agent: {e}")
        return {
            'status': 'error',
            'message': str(e)
        }

# Prepare the Strands Agent
print("🤖 Preparing Strands Agent...")
preparation_result = prepare_strands_agent()

if preparation_result['status'] == 'prepared':
    print("\n🎉 Agent is ready for testing!")
    print("💡 You can now:")
    print("   1. Test the agent in AWS Bedrock console")
    print("   2. Use the agent via API calls")
    print("   3. Try the sample queries from Step 11")
elif preparation_result['status'] == 'timeout':
    print("\n⏳ Agent preparation is still in progress")
    print("💡 Check the AWS Bedrock console for current status")
else:
    print(f"\n❌ Agent preparation failed: {preparation_result.get('message', 'Unknown error')}")


##Step: Agent with Gateway Integration

In [ ]:
def test_strands_agent_with_gateway():
    """Test Strands Agent that uses AgentCore Gateway tools."""
    
    if agent_config['status'] == 'demo' or gateway_config['status'] == 'demo':
        print("⚠️  Agent or Gateway is in demo mode")
        return
    
    try:
        # Import Strands components (if available)
        print("🤖 Testing Strands Agent with Gateway integration...")
        
        # This would use the Strands SDK to connect to the gateway
        # The agent would automatically use the gateway tools via MCP
        
        test_queries = [
            "Get the current stock price for Apple (AAPL) using the stock data tool",
            "Search for recent Tesla news using the news search tool", 
            "Query Amazon's 10-K report about revenue using the knowledge base tool"
        ]
        
        for i, query in enumerate(test_queries, 1):
            print(f"\n🔍 Test Query {i}: {query}")
            print("   → This would be sent to the Strands Agent")
            print("   → Agent would call AgentCore Gateway via MCP")
            print("   → Gateway would route to appropriate Lambda/KB tool")
            print("   → Results would be returned through the chain")
            
    except Exception as e:
        print(f"❌ Strands Agent testing error: {e}")

# Test Strands Agent with Gateway
test_strands_agent_with_gateway()


In [ ]:
# Test all tools through AgentCore Gateway with MCP protocol
print("🚪 Testing AgentCore Gateway with MCP protocol...")
test_gateway_tools_with_mcp()


## Summary: Complete Investment Research AgentCore Gateway V2

### 🎉 Implementation Complete!

You have successfully created a complete Investment Research system using AgentCore Gateway architecture with the following components:

### 🏗️ Architecture Overview
```
Strands Agent (Supervisor)
         ↓
  AgentCore Gateway
         ↓
┌─────────────────────────────────────────┐
│ 1. Alpha Vantage API (Marketplace)      │
│ 2. yfinance Lambda (Stock + News)       │
│ 3. Bedrock KB (Amazon 10-K Reports)     │
└─────────────────────────────────────────┘
```

### 📋 Components Created

1. **Financial Tools Lambda Function**
   - Stock data lookup using yfinance
   - Financial news search via web scraping
   - Deployed with proper IAM roles

2. **Alpha Vantage Integration**
   - Professional financial data API
   - Real-time and historical stock data
   - Technical indicators and fundamentals

3. **Bedrock Knowledge Base**
   - Amazon 10-K reports repository
   - Vector search capabilities
   - Structured financial document access

4. **Amazon Cognito Authentication**
   - User pool with OAuth domain
   - M2M client credentials flow
   - JWT token-based security

5. **AgentCore Gateway**
   - MCP-compliant tool definitions
   - JWT authentication integration
   - Lambda target configuration

6. **Strands Agent**
   - Financial research assistant
   - Multi-tool coordination
   - Claude 3 Sonnet foundation model

### 🔧 Key Features

- **Multi-Source Data**: Combines real-time market data, news, and regulatory filings
- **Secure Access**: JWT-based authentication with Cognito
- **Scalable Architecture**: Serverless Lambda functions with proper IAM roles
- **MCP Compliance**: Standard protocol for tool integration
- **Comprehensive Analysis**: Stock metrics, news sentiment, and fundamental data

### 🚀 Next Steps

1. **Test the System**: Use the Bedrock console to invoke the Strands Agent
2. **Add Alpha Vantage Key**: Replace demo key with your actual API key
3. **Upload 10-K Reports**: Populate the Knowledge Base with financial documents
4. **Customize Tools**: Extend Lambda function with additional financial metrics
5. **Monitor Performance**: Set up CloudWatch logging and metrics

### 📊 Sample Queries to Test

- "Analyze Apple's stock performance and recent news"
- "Compare Tesla and Ford stock volatility over the past month"
- "What are Amazon's key risk factors from their latest 10-K filing?"
- "Find news about semiconductor stocks and get price data for NVDA"

**🎯 Your Investment Research AgentCore Gateway V2 is ready for comprehensive financial analysis!**